In [1]:
#%%capture
%run ../dca/input/Format.ipynb
import ROOT as root
from array import array
root.gErrorIgnoreLevel = root.kFatal
%jsroot on

/home/yoren/.local/lib/python3.10/site-packages/nbformat/__init__.py:96: MissingIDFieldWarning: Cell is missing an id field, this will become a hard error in future nbformat versions. You may want to use `normalize()` on your notebooks before validations (available since nbformat 5.1.4). Previous versions of nbformat are fixing this issue transparently, and will stop doing so in the future.
  validate(nb)


Welcome to JupyROOT 6.30/06


Error in <TUnixSystem::FindDynamicLibrary>: input/logo/PHENIXTools/lib/libLogoPainter.so does not exist in /home/yoren/bnl/ROOT/install/lib:.:/home/yoren/bnl/ROOT/install/lib:/lib/x86_64-linux-gnu/glibc-hwcaps/x86-64-v3:/lib/x86_64-linux-gnu/glibc-hwcaps/x86-64-v2:/lib/x86_64-linux-gnu/tls/haswell/x86_64:/lib/x86_64-linux-gnu/tls/haswell:/lib/x86_64-linux-gnu/tls/x86_64:/lib/x86_64-linux-gnu/tls:/lib/x86_64-linux-gnu/haswell/x86_64:/lib/x86_64-linux-gnu/haswell:/lib/x86_64-linux-gnu/x86_64:/lib/x86_64-linux-gnu:/usr/lib/x86_64-linux-gnu/glibc-hwcaps/x86-64-v3:/usr/lib/x86_64-linux-gnu/glibc-hwcaps/x86-64-v2:/usr/lib/x86_64-linux-gnu/tls/haswell/x86_64:/usr/lib/x86_64-linux-gnu/tls/haswell:/usr/lib/x86_64-linux-gnu/tls/x86_64:/usr/lib/x86_64-linux-gnu/tls:/usr/lib/x86_64-linux-gnu/haswell/x86_64:/usr/lib/x86_64-linux-gnu/haswell:/usr/lib/x86_64-linux-gnu/x86_64:/usr/lib/x86_64-linux-gnu:/lib/glibc-hwcaps/x86-64-v3:/lib/glibc-hwcaps/x86-64-v2:/lib/tls/haswell/x86_64:/lib/tls/haswell:/lib

In [2]:
####PODGON 0.8!!!!!!!!!!!!!!!!!!!!!
sub_koeff=0.968
sub_koeff_or = 0.968
skip_thermal = True
shift0 = 12
add_bg = True
iOPT = 14
keep_int = True

m_min, m_max = 0.2, 4.8
window_nbins = 20
abs_norm_cc = 0.096225 * 1000 / 3500095 / 42 / 10
abs_norm_bb = 0.001102 * 10000 / 2104191 / 42 / 10 * 3
br = 0.01#(0.06*0.06*2*2+0.06*0.16*2*2 + 0.16*0.16)/8   # for ccbar to ee though D0D0bar, D+D- , D0->e+ anything, D+-> e+ anything
abs_norm_cc_new = 5000/3.5e7/10*br * 3
print(f"BR for ccbar to ee: {br:.2E}, keff = {abs_norm_cc_new:.2E}, old keff = {abs_norm_cc:.2E}, bb keff = {abs_norm_bb:.2E}")
print(f"Ratio new/old for ccbar: {abs_norm_cc_new/abs_norm_cc:.2f}, old cross section {96} mb, new : {96*abs_norm_cc_new/abs_norm_cc:.2f} mb, bbbar old cross section {1.102} mb, new : {1.102* 2:.2f} mb")


BR for ccbar to ee: 1.00E-02, keff = 4.29E-07, old keff = 6.55E-08, bb keff = 3.74E-08
Ratio new/old for ccbar: 6.55, old cross section 96 mb, new : 628.54 mb, bbbar old cross section 1.102 mb, new : 2.20 mb


In [3]:

keff_bg0 = 0.03  # your current BG residual factor

keffs = [1,1.,1,1,10000/3.6e7,0.10e-7,1,1,1.5e-7,1,1,1,1]
keffs = [1,1.3,1,1.3,abs_norm_cc_new,abs_norm_bb,1.3,1,1.5e-7,1,1.3,1,1,1,1]

In [4]:
component_cfg = {
    "thermal": {
        "isim": 8,
        "m_min": m_min,
        "m_max": 5.0,
        "lambda_prior": 2.0,
        "k_min": 0.1,
        "k_max": 5.0,
    },
    "ccbar": {
        "isim": 4,
        "m_min": m_min,
        "m_max": 5.0,
        "lambda_prior": 2.0,  # can start quite strong if you want small deviations
        "k_min": 0.5,
        "k_max": 1.5,
    },
    "bbbar": {
        "isim": 5,
        "m_min": m_min,
        "m_max": 5.0,
        "lambda_prior": 2.0,
        "k_min": 0.5,
        "k_max": 1.5,
    },
    "phi": {
        "isim": 1,
        "m_min": 0.8,
        "m_max": 1.1,
        "lambda_prior": 5e11,   # stronger prior, narrow region; for not using in fit lambda should be set very high
        "k_min": 0.9,
        "k_max": 1.1,
    },
    "jpsi": {
        "isim": 3,
        "m_min": 2.6,
        "m_max": 3.2,
        "lambda_prior": 5e11,
        "k_min": 0.9,
        "k_max": 1.1,
    },
    
}


In [5]:
iOption0 = 10

In [6]:
write_html = False
save_to_pdf = False
print_logo = False

In [7]:
centralities = ["0%-93%","0%-40%","20-40%","40-60%","60-93%","40%-93%"]
Opt_names = ["std", "basic","tight", "std"]
Hists_names = ["m_{ee}_stat","m_{ee}_syst","dca_stat","dca_syst","dca_{J/#Psi}_stat","dca_{J/#Psi}_syst"]
part_names = ["#pi^{0} #rightarrow ee#gamma","#phi #rightarrow ee & #etaee","#phi #rightarrow #etaee","J/#psi  #rightarrow ee",\
              "c#bar{c} #rightarrow ee","b#bar{b} #rightarrow ee","#omega #rightarrow ee & #pi^{0}ee","#omega #rightarrow #pi^{0}ee","QGP",\
              "#eta/#eta' #rightarrow ee#gamma", "#rho #rightarrow ee","#eta' #rightarrow ee#gamma","#psi(2S) #rightarrow ee","#pi^{-}","#gamma #rightarrow ee","kek"]

In [8]:
N_centr = 6
N_options = 6
N_pt = 1
colors0=[1,2,root.kGreen+2,4,root.kMagenta,root.kOrange+4,root.kGray,root.kGray,root.kBlack,root.kYellow,root.kViolet,root.kCyan,root.kGreen,root.kOrange]
colors=[2,4,4,root.kGreen+2,root.kOrange+4,root.kMagenta,root.kGray,root.kGray,root.kBlack,root.kCyan,root.kOrange,root.kCyan,root.kGreen,root.kViolet,root.kOrange]
central_bins = [0,20,60,80,20,40,40,60,60,80,80,100]
pt_boarders = [0.4,0.7,1.0,2.0,0,5.0,1.0,5.0,0.5,5]
file_path="../dca/input/data/"
file_names=["m_ee_DCA_60.root","m_ee_DCA_60.root","m_ee_DCA_30.root","m_ee_DCA_12.root","m_ee_DCA_11.root","m_ee_DCA_4.root"]
sim_file_path="../sim/output/spring26/dca/"
sim_file_names=['pi0_gee_v00.root', 'phi_ee_v00.root', "phi_ee_v00.root", 'jpsi_ee_v00.root', 'ccbar_soft_v00.root', 'ccbar_soft_v00.root',\
                 'ccbar_soft_v00.root', 'ccbar_soft_v00.root', 'ccbar_soft_v00.root',  'ccbar_soft_v00.root', 'ccbar_soft_v00.root','ccbar_soft_v00.root','ccbar_soft_v00.root', 'ccbar_soft_v00.root']#thermal_25M_200_v0
file_order = [0,9, 10, 6, 7, 1, 2, 3, 4, 5, 8]
hist_names = ["inv_mass_stat","inv_mass_syst","dca_stat_0","dca_syst_0","dca_stat_1","dca_syst_1","inv_mass_FG","inv_mass_BG","reb_ratio_FG_BG","syst_reb_ratio_FG_BG","dca_BG_0"]
hist_select_3D_names = ["inv_mass_dca_fg0_6","inv_mass_dca_fg0_5","inv_mass_dca_fg0_6","inv_mass_dca_fg0_7","inv_mass_dca_fg0_8","inv_mass_dca_fg0_9",\
                        "inv_mass_dca_fg2_6","inv_mass_dca_fg2_5","inv_mass_dca_fg2_6","inv_mass_dca_fg2_7","inv_mass_dca_fg2_8","inv_mass_dca_fg2_9",\
                        "inv_mass_dca_fg4_6","inv_mass_dca_fg4_5","inv_mass_dca_fg4_6","inv_mass_dca_fg4_7","inv_mass_dca_fg4_8","inv_mass_dca_fg4_9",\
                        "inv_mass_dca_gen_6","inv_mass_dca_gen_5","inv_mass_dca_gen_6","inv_mass_dca_gen_7","inv_mass_dca_gen_8","inv_mass_dca_gen_9"]
N_type = len(hist_names)
N_hists = len(hist_select_3D_names)

In [9]:
dNdy = [41.4,0.421,0.759/1000.,0.12,0.01,3.65,10,0.133/1000]
dNdy = [95./257*42.2,0.421,0.421,0.759/1000.,0.12,0.01,3.65,3.65,10,11./257*42.2,8.6*42/257,11./257*42.2,0.133/1000,1]
Ncolls = [251,771,282.4,82.6,12.1,40]
Br_to_ee = [1.174e-2, 2.97e-4, 1.08e-4, 5.94e-2, 0.01, 0.01, 7.28e-5, 7.7e-4, 1e-2, 6.9e-3, 4.73e-5, 4.73e-4, 7.9e-3,1.]
RAA = [[0.4,0.8,0.4,0.4,0.4,0.4,0.4,0.4],[0.3,0.6,0.3,0.3,0.3,0.3,0.3,0.3],[0.4,0.8,0.4,0.4,0.4,0.4,0.4,0.4],[0.8,1.2,0.8,0.8,0.8,0.8,0.8,0.8],[0.9,1.0,0.9,0.9,0.9,0.9,0.9,0.9],[0.9,1.1,0.8,0.9,0.9,0.9,0.9,0.9]]
dNdy_centr = [[dNdy[i]*Ncolls[j]/42.2 for i in range(len(Br_to_ee))] for j in range(6)]

In [10]:
def copycat_hist(hist,seed=0):
    seed += root.gRandom.Integer(10000)
    new_hist = root.TH3D(hist.GetName()+f"{seed}", hist.GetTitle()+f"{seed}", hist.GetNbinsX(), hist.GetXaxis().GetBinLowEdge(1), hist.GetXaxis().GetBinUpEdge(hist.GetNbinsX()),\
        hist.GetNbinsY(), hist.GetYaxis().GetBinLowEdge(1), hist.GetYaxis().GetBinUpEdge(hist.GetNbinsY()),\
        hist.GetNbinsZ(), hist.GetZaxis().GetBinLowEdge(1), hist.GetZaxis().GetBinUpEdge(hist.GetNbinsZ()))
    for ix in range(1, hist.GetNbinsX()+1):
        for iy in range(1, hist.GetNbinsY()+1):
            for iz in range(1, hist.GetNbinsZ()+1):
                new_hist.SetBinContent(ix, iy, iz, hist.GetBinContent(ix, iy, iz))
    return new_hist

In [11]:
hists_read = []

for iFile in range(len(file_names)):
    infile = root.TFile.Open(file_path+file_names[iFile], "read")
    hists_emc_read0 = []
    for itype in range(N_type):
        hist_select_3D0 = []
        for icentr in range(N_centr):
            hist_select_3D0.append(infile.Get(hist_names[itype]+f"_{icentr}{0}"))
            try: hist_select_3D0[-1].SetDirectory(root.nullptr)
            except: 
                print(f"Error setting directory for {hist_names[itype]}_{icentr}{0} in file {file_names[iFile]}")
                break
            hist_select_3D0[-1].SetName(hist_select_3D0[-1].GetName()+f"{iFile}")
        hists_emc_read0.append(hist_select_3D0)
    hists_read.append(hists_emc_read0)

    infile.Close()

hists_sim = []

for iFile in range(len(sim_file_names)):
    infile = root.TFile.Open(sim_file_path+sim_file_names[iFile], "read")
    hists_sim_file = []
    for ihist in range(N_hists):
        hists_sim_file.append(infile.Get(hist_select_3D_names[ihist]))
        hists_sim_file[-1].SetDirectory(root.nullptr)
        hists_sim_file[-1].SetName(hists_sim_file[-1].GetName()+f"_sim_{iFile}")
        if ihist%6==0:hists_sim_file[-1].SetName(hists_sim_file[-1].GetName()+f"_sim_{iFile}_{1}")
        if ihist%6==5:hists_sim_file[-1].SetName(hists_sim_file[-1].GetName()+f"_sim_{iFile}_{0}")
    hists_sim.append(hists_sim_file)
    infile.Close()

Error setting directory for dca_stat_1_00 in file m_ee_DCA_60.root
Error setting directory for dca_syst_1_00 in file m_ee_DCA_60.root
Error setting directory for dca_stat_1_00 in file m_ee_DCA_60.root
Error setting directory for dca_syst_1_00 in file m_ee_DCA_60.root
Error setting directory for dca_stat_1_00 in file m_ee_DCA_4.root
Error setting directory for dca_syst_1_00 in file m_ee_DCA_4.root


In [12]:
edges = array('d', [
    0.,0.025,0.05,0.075,0.1,0.15,0.25,0.35,0.45,0.55,0.625,0.7,0.775,0.8,0.9,0.95,1.0,1.05,
    1.2,1.5,1.75,2.0,2.25,2.5,2.75,2.9,3.0,3.05,3.1,3.15,3.2,3.3,3.45,
    3.55,3.65,3.75,3.85,4.5
])



edges = array('d', [
    0.,0.025,0.05,0.075,0.1,0.15,0.2,0.25,0.3,0.35,0.4,0.45,0.5,0.55,0.6,0.65,0.675,0.725,0.750,0.775,0.8,0.9,0.95,0.975,1.0,1.025,1.1,
    1.2,1.4,1.5,1.75,2.0,2.25,2.5,2.75,2.9,3.0,3.05,3.1,3.15,3.2,3.3,3.45,
    3.55,3.65,3.75,3.85,4.5
])

nbins = len(edges) - 1

In [13]:
hist_names3d = [f"inv_mass_ee_DCA_V{iOPT}_FG12",f"inv_mass_ee_DCA_V{iOPT}_BG12","PoolStatistics"]
hists_read_3d = []

for iFile in range(len(file_names)):
    infile = root.TFile.Open(file_path+file_names[iFile], "read")
    hists_emc_read0 = []
    for itype in range(len(hist_names3d)):
        hists_emc_read0.append(infile.Get(hist_names3d[itype]))#+f"_{icentr}{0}"
        try: hists_emc_read0[-1].SetDirectory(root.nullptr)
        except: 
            print(f"Error setting directory for {hist_names3d[itype]} in file {file_names[iFile]}")
            break
        hists_emc_read0[-1].SetName(hists_emc_read0[-1].GetName()+f"{iFile}")
    hists_read_3d.append(hists_emc_read0)

    infile.Close()

Error setting directory for inv_mass_ee_DCA_V14_FG12 in file m_ee_DCA_30.root
Error setting directory for inv_mass_ee_DCA_V14_FG12 in file m_ee_DCA_12.root
Error setting directory for inv_mass_ee_DCA_V14_FG12 in file m_ee_DCA_11.root
Error setting directory for inv_mass_ee_DCA_V14_FG12 in file m_ee_DCA_4.root


In [14]:
sim_qgp = hists_sim[8][0+shift0].ProjectionY(hists_sim[8][0+shift0].GetName()+f"_proj{8}_keks",1,80,1,25)
sim_ccb = hists_sim[4][0+shift0].ProjectionY(hists_sim[4][0+shift0].GetName()+f"_proj{4}_keks",1,80,1,25)
low_bin = sim_ccb.FindBin(1.2); high_bin = sim_ccb.FindBin(2.6)
keff_adjust = sim_qgp.Integral(low_bin,high_bin)*keffs[8]/(sim_ccb.Integral(low_bin,high_bin)*keffs[4]+1e-24)
print(f"keff_adjust = {keff_adjust}")
keffs[8] = keffs[8]/keff_adjust

keff_adjust = 0.35


In [15]:
#checking DCA projections in m_ee bins this method vs other code : all good!
rebin = 1
keff = 0.03
iOpt = 0
m_ee_low = 1.05
m_ee_high = 2.6
shift = shift0
sim_dca = [1,3,4,5,8]#phi, jpis, ccbar, bbar, thermal
c3 = root.TCanvas(f"c2",f"c2",900,900)
c3.Divide(1,1)
legends =[]
proj_mass = hists_read_3d[0][0].ProjectionX(f"inv_mass_for_bin_centr_{0}{0}")
bin_low, bin_up = proj_mass.FindBin(m_ee_low), proj_mass.FindBin(m_ee_high)
new_DCA_bins = array( 'd', [0,25,50,75,100,125,150,175,225,275,375,475,750,1000])
for icentr in range(1):
    maxim = 0
    for jtype in range(1):
        
        do_jpsi_m = jtype
        pt_low_loc, pt_high_loc = 1, 12
        phi_v_low_loc, phi_v_high_loc = bin_low, bin_up
        #print(inv_m_bins[phi_v_low-1],inv_m_bins[phi_v_high])
        c3.cd(icentr+1)
        proj = hists_read_3d[iOpt][0].ProjectionY(f"dca_stat_{jtype}_{icentr}{iOpt}",phi_v_low_loc,phi_v_high_loc,pt_low_loc, pt_high_loc)
        proj_syst = hists_read_3d[iOpt][0].ProjectionY(f"dca_syst_{jtype}_{icentr}{iOpt}",phi_v_low_loc,phi_v_high_loc,pt_low_loc, pt_high_loc)
        proj_BG = hists_read_3d[iOpt][1].ProjectionY(f"dca_BG_{jtype}_{icentr}{iOpt}",phi_v_low_loc,phi_v_high_loc,pt_low_loc, pt_high_loc)
        proj_BG.Scale(1.0)


        proj = proj.Rebin(len(new_DCA_bins)-1,proj.GetName()+"_reb",new_DCA_bins)
        proj_syst = proj_syst.Rebin(len(new_DCA_bins)-1,proj_syst.GetName()+"_reb",new_DCA_bins)
        proj_BG = proj_BG.Rebin(len(new_DCA_bins)-1,proj_BG.GetName()+"_reb",new_DCA_bins)

        for ibin in range (1,1+proj.GetNbinsX()):
            proj.SetBinError(ibin,abs(proj.GetBinContent(ibin))**0.5)
            proj_BG.SetBinError(ibin,abs(proj_BG.GetBinContent(ibin))**0.5)
            proj.SetBinContent(ibin,proj.GetBinContent(ibin)/proj.GetBinWidth(ibin))
            proj_BG.SetBinContent(ibin,proj_BG.GetBinContent(ibin)/proj_BG.GetBinWidth(ibin))
            proj.SetBinError(ibin,proj.GetBinError(ibin)/proj.GetBinWidth(ibin))
            proj_BG.SetBinError(ibin,proj_BG.GetBinError(ibin)/proj_BG.GetBinWidth(ibin))

        Format_Hist_total(proj,"DCA, #mum","dN/DCA, #mum^{-1}",0.17,0.13,0.01,0.01,0.07,0.06,21,1.5,4-jtype*3,3,4-jtype*3,0.85,1.2)

        Format_Graph(proj_BG,20,1.5,2+jtype*4,3,2+jtype*4)
        if True: 

            proj.Scale(1./hists_read_3d[iOpt][2].GetBinContent(2))
            proj_BG.Scale(sub_koeff/hists_read_3d[iOpt][2].GetBinContent(2))
            proj.Add(proj_BG,-1+keff)

        if not maxim: maxim = proj.GetBinContent(proj.FindBin(0))+proj.GetBinContent(proj.FindBin(0)+1)
        

        for ibin in range (1,1+proj.GetNbinsX()):
            proj_syst.SetBinContent(ibin,proj.GetBinContent(ibin))
            proj_syst.SetBinError(ibin,proj_BG.GetBinContent(ibin)/100.)

        root.gPad.SetLogy(1)
        if jtype==0: proj.Draw("p")
        else: proj.Draw("same p")
        hists_read[iOpt][2][0].GetXaxis().SetRangeUser(0,1000)
        hists_read[iOpt][2][0].GetYaxis().SetRangeUser(2e-14,9e-8)
        hists_read[iOpt][2][0].SetMarkerColor(6); hists_read[iOpt][2][0].SetMarkerStyle(20); hists_read[iOpt][2][0].SetMarkerSize(1.)
        hists_read[iOpt][2][0].Draw("same p")
        proj_syst.SetFillColorAlpha(4-jtype*3,0.4)
        proj_syst.Draw("same e2")
        proj.SetLineWidth(3)
        #print(proj.GetMean(),proj.GetRMS(),proj_BG.GetMean(),proj_BG.GetRMS())
        proj_BG.SetLineWidth(3)
        proj_BG.SetLineColor(2)
        proj.SetAxisRange(-2000,2000)
        proj.SetAxisRange(0,900)
        if keff > 0:
            proj_BG.Scale(keff)
            proj_BG.Draw("same p")
        proj.GetXaxis().SetNdivisions(505)
        proj.GetYaxis().SetNdivisions(50510)
        if jtype == 0:
            legends.append(root.TLegend(0.55,0.6,0.95,0.95,centralities[icentr]))
            Format_Legend(legends[-1],0.08)
        legends[icentr].AddEntry(proj,f"{proj_mass.GetBinCenter(bin_low):.2f}"+"<m_{inv}<"+f"{proj_mass.GetBinCenter(bin_up):.2f}","p")
        if keff > 0:
            legends[icentr].AddEntry(proj,f"{proj_mass.GetBinCenter(bin_low):.2f}"+"<m_{inv}<"+f"{proj_mass.GetBinCenter(bin_up):.2f}","p")
        legends[icentr].Draw()


        
        sum_sim_proj = hists_sim[0][icentr].ProjectionX(hists_sim[len(sim_file_names)-1][icentr].GetName()+"_proj1_dca_tot_new") 
        sum_sim_proj.Add(sum_sim_proj,-1)
        sim_bin_low = hists_sim[0][icentr].GetYaxis().FindBin(m_ee_low)
        sim_bin_up = hists_sim[0][icentr].GetYaxis().FindBin(m_ee_high)
        for isim in sim_dca:
            if skip_thermal and isim == 8: continue
            sim_proj = hists_sim[isim][icentr].ProjectionX(hists_sim[isim][icentr].GetName()+"_proj1_new_dca",sim_bin_low,sim_bin_up,1,25) 
            sim_proj.Scale(0.)
            h_stat = copycat_hist(hists_sim[isim][icentr+18])
            h_stat.Scale(0.)
            for jsim in range(1,6):
                sim_proj.Add(hists_sim[isim][shift+jsim].ProjectionX(hists_sim[isim][shift+jsim].GetName()+f"_proj1_new_{isim}{iOpt}",sim_bin_low,sim_bin_up,1,25),
                             10./sim_proj.GetBinWidth(1)*keffs[isim])
                h_stat.Add(hists_sim[isim][jsim+18])
            sim_proj.Scale(1./h_stat.GetEntries())
            #sim_proj.Scale(10./sim_proj.GetBinWidth(1)/hists_sim[isim][icentr+18].GetEntries()*keffs[icentr][isim])
            Format_Graph(sim_proj,21+isim,0,colors[isim],3,colors[isim])
            sum_sim_proj.Add(sim_proj,1)
            sim_proj.Draw("same H")
            legends[-1].AddEntry(sim_proj,part_names[isim],"l")
        
        if add_bg:
            Format_Graph(hists_read[iOpt][10][icentr],21,1.5,root.kCyan,3,root.kCyan)
            hists_read[iOpt][10][icentr].Draw("same")
            #sum_sim_proj.Add(hists_read[iOpt][9][icentr],1)
            for ibin in range(1,sum_sim_proj.GetNbinsX()+1):
                jbin = hists_read[iOpt][10][icentr].FindBin(sum_sim_proj.GetBinCenter(ibin))
                bin_additive= hists_read[iOpt][10][icentr].GetBinContent(jbin)#/hists_read[iOpt][10][icentr].GetBinWidth(jbin)*sum_sim_proj.GetBinWidth(ibin)
                sum_sim_proj.SetBinContent(ibin,sum_sim_proj.GetBinContent(ibin)+bin_additive)
        Format_Graph(sum_sim_proj,21,0,2,3,2)
        for ibin in range(1,sum_sim_proj.GetNbinsX()+1):
            sum_sim_proj.SetBinError(ibin, 0.1*sum_sim_proj.GetBinContent(ibin))
        sum_sim_proj.Draw("same H")

c3.Draw()

In [16]:
import numpy as np

# configuration
rebin = 1
pt_low_loc_data, pt_high_loc_data = 1, 12
pt_low_loc_sim,  pt_high_loc_sim  = 1, 25

# phi and J/psi allowed mass ranges
phi_lo,  phi_hi  = 0.5, 1.05
jpsi_lo, jpsi_hi = 2.6, 3.2

# indexes of templates in hists_sim
sim_dca = [1,3,4,5,8]  # phi, jpsi, ccbar, bbar, thermal

# centrality (you currently loop only icentr in range(1))
icentr = 0

# Get mass axis (data)
proj_mass = hists_read_3d[iOpt][0].ProjectionX("inv_mass_for_bins_tmp")
nbins_mass = proj_mass.GetNbinsX()

# helper: build sliding windows in mass-bin indices and their mass centers
mass_windows = []
h3_data = hists_read_3d[0][0]          # data TH3
ax_m_data = h3_data.GetXaxis()         # assume X is m_ee

h3_data = hists_read_3d[0][0]
ax_m_data = h3_data.GetXaxis()

first_bin = ax_m_data.FindBin(m_min)
last_bin_for_center = ax_m_data.FindBin(m_max)

mass_windows = []
for ibin_start in range(first_bin,
                        last_bin_for_center - window_nbins + 2):
    ibin_end = ibin_start + window_nbins - 1

    m_lo = ax_m_data.GetBinLowEdge(ibin_start)
    m_hi = ax_m_data.GetBinUpEdge(ibin_end)
    m_ctr = 0.5 * (m_lo + m_hi)

    # optional: clamp to [m_min, m_max] exactly
    m_lo = max(m_lo, m_min)
    m_hi = min(m_hi, m_max)

    mass_windows.append((m_lo, m_hi, m_ctr))



In [17]:
def make_data_dca_hist(m_lo, m_hi,
                       icentr=0,
                       iOpt=0,
                       rebin=1,
                       pt_low=1, pt_high=12,
                       keff_bg=0.03,
                       sub_koeff=sub_koeff,
                       nev_bin=2):
    """
    Build DCA 1D hist for data in a given mass window [m_lo, m_hi],
    using the same logic as your DCA macro:

      - project FG and BG from hists_read_3d
      - rebin
      - convert to per-µm (divide by bin width)
      - scale by 1/N_events (hists_read_3d[iOpt][2].GetBinContent(nev_bin))
      - subtract (sub_koeff - keff_bg) * BG, i.e. FG - 0.97 * BG for sub_koeff=1, keff_bg=0.03

    Returns: TH1D with FG - 0.97*BG normalized per event and per µm.
    """

    # 3D histograms: [0]=FG, [1]=BG, [2]=N_events vs centrality
    h3_fg = hists_read_3d[iOpt][0]
    h3_bg = hists_read_3d[iOpt][1]

    # mass axis is X
    ax_m = h3_fg.GetXaxis()
    bin_low = ax_m.FindBin(m_lo)
    bin_up  = ax_m.FindBin(m_hi)

    # protect edges
    if m_lo <= ax_m.GetBinLowEdge(1):
        bin_low = 1
    if m_hi >= ax_m.GetBinUpEdge(ax_m.GetNbins()):
        bin_up = ax_m.GetNbins()

    # project DCA (Y axis), integrate pT (Z axis) over [pt_low, pt_high]
    name_fg = f"dca_stat_{icentr}_{m_lo:.3f}_{m_hi:.3f}"
    name_bg = f"dca_BG_{icentr}_{m_lo:.3f}_{m_hi:.3f}"

    proj    = h3_fg.ProjectionY(name_fg, bin_low, bin_up, pt_low, pt_high)
    proj_bg = h3_bg.ProjectionY(name_bg, bin_low, bin_up, pt_low, pt_high)

    proj.Rebin(rebin)
    proj_bg.Rebin(rebin)

    # per-bin Poisson errors + divide by bin width (per µm)
    for ibin in range(1, proj.GetNbinsX()+1):
        bw = proj.GetBinWidth(ibin)

        c_fg = proj.GetBinContent(ibin)
        e_fg = (abs(c_fg))**0.5

        c_bg = proj_bg.GetBinContent(ibin)
        e_bg = (abs(c_bg))**0.5

        # convert to per µm
        if bw > 0:
            proj.SetBinContent(ibin, c_fg / bw)
            proj.SetBinError(ibin,   e_fg / bw)

            proj_bg.SetBinContent(ibin, c_bg / bw)
            proj_bg.SetBinError(ibin,   e_bg / bw)

    # 1/N_events normalization (same as your code)
    nev = hists_read_3d[iOpt][2].GetBinContent(nev_bin)
    if nev > 0:
        proj.Scale(1.0/nev)
        proj_bg.Scale(sub_koeff/nev)  # this is the BG used for subtraction

    # subtract (1 - keff_bg)*BG  → for sub_koeff=1 this is FG - 0.97*BG
    proj.Add(proj_bg, -1.0 + keff_bg)

    # optional: style in your way here if you want
    # Format_Hist_total(...)

    proj.SetDirectory(0)
    return proj


In [18]:
def make_sim_dca_templates(m_lo, m_hi,
                           icentr=0,
                           sim_dca=None,
                           shift=shift0,
                           rebin=1,
                           pt_low=1, pt_high=25,
                           keffs=None,
                           skip_thermal=False):
    """
    Build DCA templates for simulation components in sim_dca
    for a given mass window [m_lo, m_hi], using the same logic as your macro:

      - project DCA (X) in [m_lo, m_hi] on Y axis (mass axis in sim is Y)
      - sum 5 subsamples (jsim=1..5) with scale factor 10/ΔDCA * keffs[isim]
      - normalize by total h_stat entries
      - rebin
      - (optionally skip thermal)

    Returns: dict {part_names[isim]: TH1D}
    """

    if sim_dca is None:
        # default: use your global list
        sim_dca = [1, 3, 4, 5, 8]

    if keffs is None:
        keffs = [1,2,1,1,1e-7,0.10e-7,1,1,1.5e-7,1,1,1]

    templates = {}

    # we use one of the components just to get the mass axis
    ref_isim = sim_dca[0]
    h3_ref = hists_sim[ref_isim][icentr]
    ax_m_sim = h3_ref.GetYaxis()  # in sim: Y = mass

    sim_bin_low = ax_m_sim.FindBin(m_lo)
    sim_bin_up  = ax_m_sim.FindBin(m_hi)

    for isim in sim_dca:
        if skip_thermal and isim == 8:
            continue

        name_base = hists_sim[isim][icentr].GetName()
        name_proj = f"{name_base}_proj_dca_{m_lo:.3f}_{m_hi:.3f}"

        # base projection for this component
        sim_proj = hists_sim[isim][icentr].ProjectionX(
            name_proj,
            sim_bin_low, sim_bin_up,
            pt_low, pt_high
        )
        sim_proj.SetDirectory(0)
        sim_proj.Scale(0.0)

        # stats hist used to normalize
        h_stat = copycat_hist(hists_sim[isim][icentr+18])
        h_stat.Scale(0.0)

        # sum 5 subsamples as in your macro
        for jsim in range(1, 6):
            h3_sub = hists_sim[isim][shift+jsim]
            name_sub = f"{h3_sub.GetName()}_proj_dca_{isim}_{jsim}"
            sim_sub = h3_sub.ProjectionX(
                name_sub,
                sim_bin_low, sim_bin_up,
                pt_low, pt_high
            )

            # scale factor from your code: 10/ΔDCA * keffs[isim]
            bw = sim_proj.GetBinWidth(1)
            if bw <= 0:
                bw = 1.0

            scale = 10.0 / bw * keffs[isim]
            sim_proj.Add(sim_sub, scale)

            h_stat.Add(hists_sim[isim][jsim+18])

        # normalize by number of events
        if h_stat.GetEntries() > 0:
            sim_proj.Scale(1.0 / h_stat.GetEntries())

        if rebin > 1:
            sim_proj.Rebin(rebin)

        # you may also want per-µm like data:
        # for ibin in range(1, sim_proj.GetNbinsX()+1):
        #     bw = sim_proj.GetBinWidth(ibin)
        #     if bw > 0:
        #         sim_proj.SetBinContent(ibin, sim_proj.GetBinContent(ibin)/bw)

        templates[part_names[isim]] = sim_proj

    return templates


In [19]:
def make_bg03_template_dca(m_lower=1.2, m_upper=2.6, pt_lower = 1, pt_upper = 12, icentr=0, iOpt=0, rebin=1, scale_loc = 0.03):
    """
    Build the 0.03 * BG DCA template you add on top of the cocktail.
    Using hists_read[iOpt][10][icentr] as in your macro.
    """
    bin_low = hists_read_3d[iOpt][1].GetXaxis().FindBin(m_lower)
    bin_up = hists_read_3d[iOpt][1].GetXaxis().FindBin(m_upper)
    #print(f"BG DCA template mass bins: {bin_low}-{bin_up}")
    h_bg1d = hists_read_3d[iOpt][1].ProjectionY(hists_read_3d[iOpt][1].GetName()+f"_{m_lower}_{m_upper}_{scale_loc}", bin_low, bin_up, pt_lower, pt_upper)
    h_bg1d.SetDirectory(0)

    if rebin > 1:
        h_bg1d.Rebin(rebin)
        
    h_bg1d.Scale(scale_loc*sub_koeff_or/hists_read_3d[iOpt][2].GetBinContent(2)/h_bg1d.GetBinWidth(1))  # 3% BG

    return h_bg1d


In [20]:
canvases = []
legends = []
shift = shift0
ccbar_int = 0; bbbar_int = 0

for ipar in [0]:  # centrality index offset
    c0 = root.TCanvas(f"c1_{ipar}",f"c1_{ipar}",1000,1000)
    c0.Divide(1,1)
    canvases.append(c0)
    iOpt = 0
    rebin = 2

    # base normalizations from integrals
    # IMPORTANT: thermal (isim=8) cannot be 0, otherwise k_corr won't help
    if keffs is None:
        keffs = [1,2,1,1,1e-7,0.10e-7,1,1,1.5e-7,1,1,1]

    for icentr in range(ipar,1+ipar):
        legends.append(root.TLegend(0.2,0.90,0.99,0.96,
                                    "Run 2014 "+centralities[icentr]+" Au+Au #sqrt{s_{NN}} = 200 GeV"))
        Format_Legend(legends[-1],0.04,12)
        legends[-1].Draw()

        legends.append(root.TLegend(0.2, 0.86, 0.99, 0.89,
                                    "|y^{e}|<0.35, p_{T}^{e }>0.4 GeV, p_{T}^{ee}>0.8 GeV"))
        Format_Legend(legends[-1],0.04,12)

        c0.cd(1)
        root.gPad.SetLogy()

        proj = hists_read_3d[iOpt][0].ProjectionX( hists_read_3d[iOpt][0].GetName()+f"_proj_mass{iOpt}{icentr}", 1, 40,1, 12)
        proj_syst = hists_read_3d[iOpt][0].ProjectionX( hists_read_3d[iOpt][0].GetName()+f"_proj_syst_mass{iOpt}{icentr}", 1, 40,1, 12)
        proj_BG = hists_read_3d[iOpt][1].ProjectionX( hists_read_3d[iOpt][1].GetName()+f"_proj_mass{iOpt}{icentr}", 1, 40,1, 12)

        proj    = proj.Rebin(nbins,    proj.GetName()+"proj_reb",    edges)
        proj_syst = proj_syst.Rebin(nbins, proj_syst.GetName()+"proj_syst_reb", edges)
        proj_BG = proj_BG.Rebin(nbins, proj_BG.GetName()+"proj_BG_reb", edges)
    
        for ibin in range (1,1+proj.GetNbinsX()):
            proj.SetBinError(ibin,(proj.GetBinContent(ibin))**0.5)
            keff  = sub_koeff
            if ibin > proj.FindBin(3.4): proj_BG.SetBinContent(ibin,(proj_BG.GetBinContent(ibin))*0.9999)
            proj.SetBinContent(ibin,proj.GetBinContent(ibin) - keff*proj_BG.GetBinContent(ibin)*1)
            proj.SetBinContent(ibin,proj.GetBinContent(ibin)/proj.GetBinWidth(ibin))
            proj.SetBinError(ibin,proj.GetBinError(ibin)/proj.GetBinWidth(ibin))
            proj_syst.SetBinContent(ibin,proj.GetBinContent(ibin))
            proj_BG.SetBinError(ibin,abs(proj_BG.GetBinContent(ibin))**0.5)
            proj_syst.SetBinError(ibin,((proj_BG.GetBinContent(ibin)/100./proj.GetBinWidth(ibin)*1)**2+(proj_BG.GetBinError(ibin)/proj.GetBinWidth(ibin)*1)**2)**0.5)


        if True: 
            proj.Scale(1./hists_read_3d[iOpt][2].GetBinContent(2))
            proj_syst.Scale(1./hists_read_3d[iOpt][2].GetBinContent(2))
            proj_BG.Scale(sub_koeff/hists_read_3d[iOpt][2].GetBinContent(2))


        Format_Hist_total(proj,
                          "m_{e^{+}e^{-}} (GeV)",
                          "dN/dm_{e^{+}e^{-}} (GeV^{-1}) in PHENIX acceptance",
                          0.15,0.15,0.01,0.01,0.05,0.05,
                          21,1.5,1,3,1,0.99,1.55)
        proj.SetMaximum(2e-3)
        proj.SetMinimum(1.001e-10)
        proj.Draw()
        #hists_read[iOpt][0][icentr].Draw("same")
        #hists_read[iOpt][1][icentr].SetFillColorAlpha(1,0.4)
        #hists_read[iOpt][1][icentr].Draw("same e2")
        proj_syst.SetFillColorAlpha(1,0.4); 
        proj_syst.Draw("same e2")

        legends.append(root.TLegend(0.25,0.86,0.45,0.90))
        Format_Legend(legends[-1],0.035,12)
        legends[-1].AddEntry(hists_read[iOpt][0][icentr],"data","p")
        legends[-1].Draw()

        legends.append(root.TLegend(0.06,0.75,0.65,0.86))
        Format_Hist_total(hists_read[iOpt][0][icentr],
                          "m_{e^{+}e^{-}} (GeV)",
                          "dN/dm_{e^{+}e^{-}} (GeV^{-1}) in PHENIX acceptance",
                          0.15,0.15,0.01,0.01,0.05,0.05,
                          21,1.5,1,3,1,0.99,1.55)
        Format_Legend(legends[-1],0.035,12)
        legends[-1].AddEntry(0,"p_{T}^{e }>0.3 GeV","")
        legends[-1].AddEntry(0,"|y^{e}|<0.35","")
        legends[-1].Draw()

        legends.append(root.TLegend(0.4,0.56,0.99,0.905))
        Format_Legend(legends[-1],0.035,12)

        # sum over cocktail components
        sum_sim_proj = hists_sim[len(sim_file_names)-1][icentr].ProjectionY(
            hists_sim[len(sim_file_names)-1][icentr].GetName()+f"_proj{iOpt}",
            1,80,1,25
        )
        sum_sim_proj.Scale(0)

        for isim in range(0, len(sim_file_names)-1):
            if isim in [21]:
                continue  # these sources are excluded

            sim_proj = hists_sim[isim][icentr+shift].ProjectionY(
                hists_sim[isim][icentr+shift].GetName()+f"_proj{isim}{iOpt}",
                1,80,1,25
            )
            sim_proj.Scale(0.)
            h_stat = copycat_hist(hists_sim[isim][icentr+18])
            h_stat.Scale(0.)
            h_stat.SetEntries(0)

            for ihist in range(1,6):
                h_sub = hists_sim[isim][ihist+shift].ProjectionY(
                    hists_sim[isim][ihist+shift].GetName()+f"_proj{isim}{ihist+shift}{iOpt}",
                    1,80,1,25
                )
                scale_base = 10.0 / h_sub.GetBinWidth(1) * keffs[isim] * 0.8
                sim_proj.Add(h_sub, scale_base)
                h_stat.Add(hists_sim[isim][icentr+ihist+18])

            if h_stat.GetEntries() > 0:
                sim_proj.Scale(1.0 / h_stat.GetEntries())

            # --- thermal-only mass-dependent correction
            n_bins = sim_proj.GetNbinsX()

            Format_Graph(sim_proj,21+isim,0,colors[isim],3,colors[isim], Lalpha=0.7)
            if isim > 12:
                sim_proj.Rebin(rebin)

            if not write_html:
                print(f"{h_stat.Integral()/h_stat.GetEntries()/Br_to_ee[isim]*10:.2E},",
                      f"{h_stat.Integral()*keffs[isim]/Br_to_ee[isim]/h_stat.GetEntries()*10:.2E},",
                      f"{dNdy_centr[icentr][isim]:.2E},",
                      f"{h_stat.Integral()*keffs[isim]/Br_to_ee[isim]/dNdy_centr[icentr][isim]/h_stat.GetEntries()*10*100:.0f}%,",
                      part_names[isim])
            if isim == 4: ccbar_int += sim_proj.Integral(sim_proj.FindBin(0.5), sim_proj.FindBin(3.5))
            elif isim == 5: bbbar_int += sim_proj.Integral(sim_proj.FindBin(0.5), sim_proj.FindBin(3.5))
            sum_sim_proj.Add(sim_proj,1)
            sim_proj.Draw("same H")
            legends[-1].AddEntry(sim_proj,part_names[isim],"l")

        Format_Graph(sum_sim_proj,21,0,colors[isim+1],3,colors[isim+1], Lalpha=0.7)
        sum_sim_proj.Draw("same H")
        legends[-1].AddEntry(sum_sim_proj,"sum","l")
        Format_Legend(legends[-1],0.04,12)
        legends[-1].SetNColumns(2)
        legends[-1].Draw()

        data_yield = proj.Integral(proj.FindBin(1.2), proj.FindBin(2.6), "width")
        sim_yield  = sum_sim_proj.Integral(sum_sim_proj.FindBin(1.2), sum_sim_proj.FindBin(2.6), "width")
        keff_inter_mass = data_yield/(sim_yield+1e-24)

    c0.Draw()
    if save_to_pdf:
        c0.Print(f"output/Note/cocktail_2_keff_massdep.pdf")

c0.SaveAs("output/DCA_templates/mass_res_before_fit.png")


3.43E-01, 3.43E-01, 9.28E+01, 0%, #pi^{0} #rightarrow ee#gamma
5.05E-01, 6.56E-01, 2.50E+00, 26%, #phi #rightarrow ee & #etaee
1.39E+00, 1.39E+00, 2.50E+00, 55%, #phi #rightarrow #etaee
1.06E-03, 1.37E-03, 4.51E-03, 30%, J/#psi  #rightarrow ee
4.44E+05, 1.90E-01, 7.14E-01, 27%, c#bar{c} #rightarrow ee
4.44E+05, 1.66E-02, 5.95E-02, 28%, b#bar{b} #rightarrow ee
6.10E+07, 7.92E+07, 2.17E+01, 365007614%, #omega #rightarrow ee & #pi^{0}ee
5.76E+06, 5.76E+06, 2.17E+01, 26546008%, #omega #rightarrow #pi^{0}ee
4.44E+05, 1.90E-01, 5.95E+01, 0%, QGP
6.43E+05, 6.43E+05, 1.07E+01, 5986345%, #eta/#eta' #rightarrow ee#gamma
9.38E+07, 1.22E+08, 8.36E+00, 1458983594%, #rho #rightarrow ee
9.38E+06, 9.38E+06, 1.07E+01, 87327225%, #eta' #rightarrow ee#gamma
5.62E+05, 5.62E+05, 7.91E-04, 71007477192%, #psi(2S) #rightarrow ee


python ERROR: cannot open image file "output/DCA_templates/mass_res_before_fit.png" for writing. Please check permissions.


In [21]:
canvases = []
legends = []
shift = shift0
ccbar_int = 0; bbbar_int = 0

for ipar in [0]:  # centrality index offset
    c0 = root.TCanvas(f"c1_{ipar}",f"c1_{ipar}",1000,1000)
    c0.Divide(1,1)
    canvases.append(c0)
    iOpt = 0
    rebin = 2

    # base normalizations from integrals
    # IMPORTANT: thermal (isim=8) cannot be 0, otherwise k_corr won't help
    if keffs is None:
        keffs = [1,2,1,1,1e-7,0.10e-7,1,1,1.5e-7,1,1,1]

    for icentr in range(ipar,1+ipar):
        legends.append(root.TLegend(0.2,0.90,0.99,0.96,
                                    "Run 2014 "+centralities[icentr]+" Au+Au #sqrt{s_{NN}} = 200 GeV"))
        Format_Legend(legends[-1],0.04,12)
        legends[-1].Draw()

        legends.append(root.TLegend(0.2, 0.86, 0.99, 0.89,
                                    "|y^{e}|<0.35, p_{T}^{e }>0.4 GeV, p_{T}^{ee}>0.8 GeV"))
        Format_Legend(legends[-1],0.04,12)

        c0.cd(1)
        root.gPad.SetLogy()

        proj = hists_read_3d[iOpt][0].ProjectionX( hists_read_3d[iOpt][0].GetName()+f"_proj_mass{iOpt}{icentr}", 1, 40,1, 12)
        proj_syst = hists_read_3d[iOpt][0].ProjectionX( hists_read_3d[iOpt][0].GetName()+f"_proj_syst_mass{iOpt}{icentr}", 1, 40,1, 12)
        proj_BG = hists_read_3d[iOpt][1].ProjectionX( hists_read_3d[iOpt][1].GetName()+f"_proj_mass{iOpt}{icentr}", 1, 40,1, 12)

        proj    = proj.Rebin(nbins,    proj.GetName()+"proj_reb",    edges)
        proj_syst = proj_syst.Rebin(nbins, proj_syst.GetName()+"proj_syst_reb", edges)
        proj_BG = proj_BG.Rebin(nbins, proj_BG.GetName()+"proj_BG_reb", edges)
    
        for ibin in range (1,1+proj.GetNbinsX()):
            proj.SetBinError(ibin,(proj.GetBinContent(ibin))**0.5)
            keff  = sub_koeff
            if ibin > proj.FindBin(3.4): proj_BG.SetBinContent(ibin,(proj_BG.GetBinContent(ibin))*0.9999)
            proj.SetBinContent(ibin,proj.GetBinContent(ibin) - keff*proj_BG.GetBinContent(ibin)*1)
            proj.SetBinContent(ibin,proj.GetBinContent(ibin)/proj.GetBinWidth(ibin))
            proj.SetBinError(ibin,proj.GetBinError(ibin)/proj.GetBinWidth(ibin))
            proj_syst.SetBinContent(ibin,proj.GetBinContent(ibin))
            proj_BG.SetBinError(ibin,abs(proj_BG.GetBinContent(ibin))**0.5)
            proj_syst.SetBinError(ibin,((proj_BG.GetBinContent(ibin)/100./proj.GetBinWidth(ibin)*1)**2+(proj_BG.GetBinError(ibin)/proj.GetBinWidth(ibin)*1)**2)**0.5)


        if True: 
            proj.Scale(1./hists_read_3d[iOpt][2].GetBinContent(2))
            proj_syst.Scale(1./hists_read_3d[iOpt][2].GetBinContent(2))
            proj_BG.Scale(sub_koeff/hists_read_3d[iOpt][2].GetBinContent(2))


        Format_Hist_total(proj,
                          "m_{e^{+}e^{-}} (GeV)",
                          "dN/dm_{e^{+}e^{-}} (GeV^{-1}) in PHENIX acceptance",
                          0.15,0.15,0.01,0.01,0.05,0.05,
                          21,1.5,1,3,1,0.99,1.55)
        proj.SetMaximum(2e1)
        proj.SetMinimum(1.001e-10)
        proj.Draw()
        #hists_read[iOpt][0][icentr].Draw("same")
        #hists_read[iOpt][1][icentr].SetFillColorAlpha(1,0.4)
        #hists_read[iOpt][1][icentr].Draw("same e2")
        proj_syst.SetFillColorAlpha(1,0.4); 
        proj_syst.Draw("same e2")

        legends.append(root.TLegend(0.25,0.86,0.45,0.90))
        Format_Legend(legends[-1],0.035,12)
        legends[-1].AddEntry(hists_read[iOpt][0][icentr],"data","p")
        legends[-1].Draw()

        legends.append(root.TLegend(0.06,0.75,0.65,0.86))
        Format_Hist_total(hists_read[iOpt][0][icentr],
                          "m_{e^{+}e^{-}} (GeV)",
                          "dN/dm_{e^{+}e^{-}} (GeV^{-1}) in PHENIX acceptance",
                          0.15,0.15,0.01,0.01,0.05,0.05,
                          21,1.5,1,3,1,0.99,1.55)
        Format_Legend(legends[-1],0.035,12)
        legends[-1].AddEntry(0,"p_{T}^{e }>0.3 GeV","")
        legends[-1].AddEntry(0,"|y^{e}|<0.35","")
        legends[-1].Draw()

        legends.append(root.TLegend(0.4,0.56,0.99,0.905))
        Format_Legend(legends[-1],0.035,12)

        # sum over cocktail components
        sum_sim_proj = hists_sim[len(sim_file_names)-1][icentr].ProjectionY(
            hists_sim[len(sim_file_names)-1][icentr].GetName()+f"_proj{iOpt}",
            1,80,1,25
        )
        sum_sim_proj.Scale(0)
        sum_h_stat_proj = hists_sim[len(sim_file_names)-1][2].ProjectionY(
            hists_sim[len(sim_file_names)-1][2].GetName()+f"_projasd1{iOpt}",
            1,80,1,25
        )
        sum_h_stat_proj.Scale(0)
        sum_h_stat_proj.Rebin(2)

        for isim in range(0, len(sim_file_names)-1):
            if isim in [21]:
                continue  # these sources are excluded
            if isim in [2,8,5,6,7,8,9,10,11,12,13,14,15,16]:
                continue  # these sources are excluded

            sim_proj = hists_sim[isim][icentr+shift].ProjectionY(
                hists_sim[isim][icentr+shift].GetName()+f"_proj{isim}{iOpt}",
                1,80,1,25
            )
            sim_proj.Scale(0.)
            h_stat = copycat_hist(hists_sim[isim][icentr+18])
            h_stat.Scale(0.)
            h_stat.SetEntries(0)
            h_stat_proj = h_stat.ProjectionY(
                h_stat.GetName()+f"_proj_thermal_massdep_corr_{isim}{iOpt}",
                1,80,1,25
            )

            for ihist in range(1,6):
                h_sub = hists_sim[isim][ihist+shift].ProjectionY(
                    hists_sim[isim][ihist+shift].GetName()+f"_proj{isim}{ihist+shift}{iOpt}",
                    1,80,1,25
                )
                scale_base = 10.0 / h_sub.GetBinWidth(1) * keffs[isim] * 0.8
                sim_proj.Add(h_sub, scale_base)
                h_stat.Add(hists_sim[isim][icentr+ihist+18])
                h_stat_proj.Add(hists_sim[isim][icentr+ihist+18].ProjectionY(
                    hists_sim[isim][icentr+ihist+18].GetName()+f"_proj_thermal_massdep_corr_sub_{isim}{ihist+shift}{iOpt}",
                    1,80,1,25
                ),10.0 / h_stat_proj.GetBinWidth(1) * keffs[isim] )

            if h_stat.GetEntries() > 0:
                sim_proj.Scale(1.0 / h_stat.GetEntries())

            # --- thermal-only mass-dependent correction
            n_bins = sim_proj.GetNbinsX()
            h_stat_proj.Scale(1.0 / h_stat.GetEntries())
            h_stat_proj.Draw("same H")
        

            Format_Graph(sim_proj,21+isim,0,colors[isim],3,colors[isim], Lalpha=0.7)
            if isim > 12:
                sim_proj.Rebin(rebin)

            if not write_html:
                print(f"{h_stat.Integral()/h_stat.GetEntries()/Br_to_ee[isim]*10:.2E},",
                      f"{h_stat.Integral()*keffs[isim]/Br_to_ee[isim]/h_stat.GetEntries()*10:.2E},",
                      f"{dNdy_centr[icentr][isim]:.2E},",
                      f"{h_stat.Integral()*keffs[isim]/Br_to_ee[isim]/dNdy_centr[icentr][isim]/h_stat.GetEntries()*10*100:.0f}%,",
                      part_names[isim])
            if isim == 4: ccbar_int += sim_proj.Integral(sim_proj.FindBin(0.5), sim_proj.FindBin(3.5))
            elif isim == 5: bbbar_int += sim_proj.Integral(sim_proj.FindBin(0.5), sim_proj.FindBin(3.5))
            sum_sim_proj.Add(sim_proj,1)
            sim_proj.Draw("same H")
            if isim not in [4,5,8]: sum_h_stat_proj.Add(sim_proj.Rebin(2),h_stat_proj.Integral(1,-1)/sim_proj.Integral(1,-1))
            else: sum_h_stat_proj.Add(h_stat_proj,1)
            legends[-1].AddEntry(sim_proj,part_names[isim],"l")

        Format_Graph(sum_sim_proj,21,0,colors[isim+1],3,colors[isim+1], Lalpha=0.7)
        sum_sim_proj.Draw("same H")
        Format_Graph(sum_h_stat_proj,21,0,colors[isim+2],3,colors[isim+2], Lalpha=0.7)
        #sum_h_stat_proj.Scale(0.5)
        sum_h_stat_proj.Draw("same H")
        legends[-1].AddEntry(sum_sim_proj,"sum","l")
        Format_Legend(legends[-1],0.04,12)
        legends[-1].SetNColumns(2)
        legends[-1].Draw()

        data_yield = proj.Integral(proj.FindBin(1.2), proj.FindBin(2.6), "width")
        sim_yield  = sum_sim_proj.Integral(sum_sim_proj.FindBin(1.2), sum_sim_proj.FindBin(2.6), "width")
        keff_inter_mass = data_yield/(sim_yield+1e-24)

    c0.Draw()
    if save_to_pdf:
        c0.Print(f"output/Note/cocktail_2_keff_massdep.pdf")

c0.SaveAs("output/DCA_templates/mass_res_before_fit.png")


3.43E-01, 3.43E-01, 9.28E+01, 0%, #pi^{0} #rightarrow ee#gamma
5.05E-01, 6.56E-01, 2.50E+00, 26%, #phi #rightarrow ee & #etaee
1.06E-03, 1.37E-03, 4.51E-03, 30%, J/#psi  #rightarrow ee
4.44E+05, 1.90E-01, 7.14E-01, 27%, c#bar{c} #rightarrow ee


python ERROR: cannot open image file "output/DCA_templates/mass_res_before_fit.png" for writing. Please check permissions.


In [22]:
c1 = root.TCanvas("c3","c3",1000,1000)
root.gPad.SetLogy()
sum_sim_proj.Draw("H")
sum_h_stat_proj.Draw("same H")
c1.Draw()

In [23]:
c1 = root.TCanvas("c3","c3",1000,1000)
root.gPad.SetLogy()
sum_sim_proj_copycat = sum_sim_proj.Clone("sum_sim_proj_copycat")
sum_h_stat_proj_copycat = sum_h_stat_proj.Clone("sum_h_stat_proj_copycat")
sum_sim_proj_copycat.Rebin(2)
sum_h_stat_proj_copycat.Rebin(1)
Format_Hist_total(sum_sim_proj_copycat,
                          "m_{e^{+}e^{-}} (GeV)",
                          "dN/dm_{e^{+}e^{-}} (GeV^{-1}) in PHENIX acceptance",
                          0.15,0.15,0.01,0.01,0.05,0.05,
                          21,1.5,1,3,1,0.99,1.55)
Format_Graph(sum_h_stat_proj_copycat,21,0,colors[isim+2],3,colors[isim+2], Lalpha=0.7)
sum_sim_proj_copycat.SetMaximum(sum_h_stat_proj_copycat.GetMaximum()*1.2)
sum_sim_proj_copycat.Draw("H")
sum_h_stat_proj_copycat.Draw("same H")
c1.Draw()

In [ ]:
c1 = root.TCanvas("c3","c3",1000,1000)
root.gPad.SetLogy()
sum_sim_proj_copycat = sum_sim_proj.Clone("sum_sim_proj_copycat")
sum_h_stat_proj_copycat = sum_h_stat_proj.Clone("sum_h_stat_proj_copycat")
sum_sim_proj_copycat.Rebin(2)
sum_h_stat_proj_copycat.Rebin(1)
Format_Hist_total(sum_sim_proj_copycat,
                          "m_{e^{+}e^{-}} (GeV)",
                          "dN/dm_{e^{+}e^{-}} (GeV^{-1}) in PHENIX acceptance",
                          0.15,0.15,0.01,0.01,0.05,0.05,
                          21,1.5,1,3,1,0.99,1.55)
Format_Graph(sum_h_stat_proj_copycat,21,0,colors[isim+2],3,colors[isim+2], Lalpha=0.7)
sum_sim_proj_copycat.Divide(sum_h_stat_proj_copycat)
for ibin in range(1, sum_sim_proj_copycat.GetNbinsX()+1):
    sum_sim_proj_copycat.SetBinError(ibin, 0.5*sum_sim_proj_copycat.GetBinContent(ibin))
sum_sim_proj_copycat.Draw("H")
sum_sim_proj_copycat.Rebin(2)
sum_sim_proj_copycat.Scale(0.5)
sum_sim_proj_copycat.Fit("pol2","R","",0.2,3.0)
print(sum_sim_proj_copycat.GetFunction("pol2").GetParameter(0),",",
      sum_sim_proj_copycat.GetFunction("pol2").GetParameter(1),",",
      sum_sim_proj_copycat.GetFunction("pol2").GetParameter(2))
#sum_h_stat_proj_copycat.Draw("same H")
c1.Draw()

#saving sum_sim_proj_copycat hist to a file for later use in DCA fit macro
out_file = root.TFile("output/sim_cocktail_massdep_corr3.root","RECREATE")
sum_sim_proj_copycat.Write()
out_file.Close()

0.0025987665435245966 , -0.005109118881802433 , 0.004443299793269407
****************************************
Minimizer is Linear / Migrad
Chi2                      =      7.43468
NDf                       =           25
p0                        =   0.00259877   +/-   0.000737572 
p1                        =  -0.00510912   +/-   0.00164375  
p2                        =    0.0044433   +/-   0.000753146 
